In [2]:
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import os

valid_image_base_path = "./ktp_images/valid_ktp/"
invalid_image_base_path = "./ktp_images/invalid_ktp/"
valid_image_paths = os.listdir(valid_image_base_path)
invalid_image_paths = os.listdir(invalid_image_base_path)

def preprocess_image(image_path, base_path, target_size=(32, 32)):
    image = cv2.imread(base_path+image_path)

    # Resize the image to the target size (e.g., 32x32)
    image = cv2.resize(image, target_size)

    # Normalize the pixel values to a range of [0, 1]
    image = image.astype('float32') / 255.0

    return image

# Assuming you have 50 valid KTP images and 50 invalid KTP images.
# Load the images and preprocess them (e.g., resize to 32x32, normalize pixel values, etc.)
# Load and preprocess the valid KTP images
valid_images = np.array([preprocess_image(image_path,valid_image_base_path) for image_path in valid_image_paths])

# Load and preprocess the invalid KTP images
invalid_images = np.array([preprocess_image(image_path,invalid_image_base_path) for image_path in invalid_image_paths])

# Let's say your valid KTP images are stored in valid_images and invalid KTP images in invalid_images.
# Combine the images into a single array and create corresponding labels (1 for valid KTP, 0 for invalid KTP).
images = np.concatenate((valid_images, invalid_images))
labels = np.concatenate((np.ones(len(valid_images)), np.zeros(len(invalid_images))))

# Shuffle the data to avoid any order bias during training
shuffle_indices = np.random.permutation(len(images))
images = images[shuffle_indices]
labels = labels[shuffle_indices]

# Split the dataset into training and testing sets (80% for training, 20% for testing)
split_idx = int(0.8 * len(images))
training_images, testing_images = images[:split_idx], images[split_idx:]
training_labels, testing_labels = labels[:split_idx], labels[split_idx:]

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(training_images, training_labels, epochs=50, batch_size=32, validation_data=(testing_images, testing_labels), callbacks=[early_stopping])

Epoch 1/50
3/3 [==============================] - 1s 109ms/step - loss: 0.7084 - accuracy: 0.5625 - val_loss: 0.6634 - val_accuracy: 1.0000
Epoch 2/50
3/3 [==============================] - 0s 38ms/step - loss: 0.6601 - accuracy: 0.6875 - val_loss: 0.6237 - val_accuracy: 1.0000
Epoch 3/50
3/3 [==============================] - 0s 22ms/step - loss: 0.6128 - accuracy: 0.9250 - val_loss: 0.5502 - val_accuracy: 1.0000
Epoch 4/50
3/3 [==============================] - 0s 19ms/step - loss: 0.5548 - accuracy: 0.8750 - val_loss: 0.4590 - val_accuracy: 1.0000
Epoch 5/50
3/3 [==============================] - 0s 26ms/step - loss: 0.5080 - accuracy: 0.7750 - val_loss: 0.3862 - val_accuracy: 1.0000
Epoch 6/50
3/3 [==============================] - 0s 28ms/step - loss: 0.4532 - accuracy: 0.7875 - val_loss: 0.3335 - val_accuracy: 1.0000
Epoch 7/50
3/3 [==============================] - 0s 24ms/step - loss: 0.3922 - accuracy: 0.9000 - val_loss: 0.2917 - val_accuracy: 1.0000
Epoch 8/50
3/3 [==========

In [4]:
loss, accuracy = model.evaluate(testing_images, testing_labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

1/1 [==============================] - 0s 19ms/step - loss: 0.0090 - accuracy: 1.0000
Loss: 0.008971523493528366
Accuracy: 1.0


In [8]:
model.save('../ktp classifier.h5/')

INFO:tensorflow:Assets written to: ../ktp classifier.h5/assets


INFO:tensorflow:Assets written to: ../ktp classifier.h5/assets
